In [ ]:
# Step 1: Install Required Libraries

In [1]:
%pip install tensorflow numpy

: 

In [2]:
# Step 2: Generate Valid Sudoku Data
# We’ll generate valid 3x3 Sudoku puzzles for training.

In [3]:
import numpy as np

def generate_valid_sudoku_grids(num_samples):
    """
    Generate valid 3x3 Sudoku grids.
    Each cell is represented as a one-hot encoded vector.
    """
    grids = []
    for _ in range(num_samples):
        # Create a valid 3x3 Sudoku grid
        grid = np.array([[1, 2, 3],
                         [3, 1, 2],
                         [2, 3, 1]])
        # Randomly permute rows and columns to generate variations
        grid = grid[np.random.permutation(3)][:, np.random.permutation(3)]
        # Flatten the grid and one-hot encode
        flattened = grid.flatten() - 1  # Convert to 0-based index
        one_hot = np.eye(3)[flattened].flatten()
        grids.append(one_hot)
    return np.array(grids)

# Generate 1000 valid Sudoku grids
sudoku_data = generate_valid_sudoku_grids(1000)
print("Sudoku data shape:", sudoku_data.shape)  # Should be (1000, 27)

Sudoku data shape: (1000, 27)


In [ ]:
# Step 3: Define and Train the First RBM
# We’ll use TensorFlow to define and train the RBMs.

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

class RBM:
    def __init__(self, visible_units, hidden_units, learning_rate=0.01):
        self.visible_units = visible_units
        self.hidden_units = hidden_units
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.W = tf.Variable(tf.random.normal([visible_units, hidden_units], stddev=0.1))
        self.v_bias = tf.Variable(tf.zeros([visible_units]))
        self.h_bias = tf.Variable(tf.zeros([hidden_units]))

    def train(self, data, epochs=10, batch_size=32):
        optimizer = tf.optimizers.Adam(learning_rate=self.learning_rate)

        for epoch in range(epochs):
            for i in range(0, len(data), batch_size):
                batch = data[i:i + batch_size]
                with tf.GradientTape() as tape:
                    # Positive phase
                    h_prob = self.sample_hidden(batch)
                    positive_grad = tf.matmul(tf.transpose(batch), h_prob)

                    # Negative phase
                    v_recon = self.sample_visible(h_prob)
                    h_recon_prob = self.sample_hidden(v_recon)
                    negative_grad = tf.matmul(tf.transpose(v_recon), h_recon_prob)

                    # Compute gradients and update weights
                    delta_W = (positive_grad - negative_grad) / tf.cast(tf.shape(batch)[0], tf.float32)
                    delta_v_bias = tf.reduce_mean(batch - v_recon, axis=0)
                    delta_h_bias = tf.reduce_mean(h_prob - h_recon_prob, axis=0)

                    grads = [delta_W, delta_v_bias, delta_h_bias]
                    optimizer.apply_gradients(zip(grads, [self.W, self.v_bias, self.h_bias]))

            print(f"Epoch {epoch + 1}/{epochs} completed")

    def sample_hidden(self, visible):
        """Sample hidden units given visible units."""
        h_prob = tf.sigmoid(tf.matmul(visible, self.W) + self.h_bias)
        return tf.nn.relu(tf.sign(h_prob - tf.random.uniform(tf.shape(h_prob))))

    def sample_visible(self, hidden):
        """Sample visible units given hidden units."""
        v_prob = tf.sigmoid(tf.matmul(hidden, tf.transpose(self.W)) + self.v_bias)
        return tf.nn.relu(tf.sign(v_prob - tf.random.uniform(tf.shape(v_prob))))

# Define and train the first RBM
rbm1 = RBM(visible_units=27, hidden_units=20, learning_rate=0.01)
rbm1.train(sudoku_data, epochs=10, batch_size=32)

InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:MatMul] name: 

In [ ]:
# Step 4: Train the Second RBM
# Use the hidden layer activations from the first RBM as input to the second RBM.

In [ ]:
# Get hidden layer activations from the first RBM
hidden_activations_1 = rbm1.sample_hidden(sudoku_data)

# Define and train the second RBM
rbm2 = RBM(visible_units=20, hidden_units=10, learning_rate=0.01)
rbm2.train(hidden_activations_1, epochs=10, batch_size=32)

In [ ]:
# Step 5: Fine-Tune the Network (Optional)
# If you have labeled data, you can fine-tune the network using supervised learning.

In [ ]:
# Add a softmax layer for classification
inputs = Input(shape=(10,))  # Output of the second RBM
outputs = Dense(3, activation='softmax')(inputs)  # 3 classes (1, 2, 3)
classifier = Model(inputs, outputs)

# Compile the classifier
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the network (assuming labeled_data is available)
# labeled_data = (X_train, y_train)
# classifier.fit(X_train, y_train, epochs=20, batch_size=32)

In [ ]:
# Step 6: Generate or Complete Sudoku Puzzles
# Use the trained RBMs to generate new Sudoku puzzles or complete partially filled ones.

In [ ]:
def generate_sudoku(rbm1, rbm2):
    """Generate a new Sudoku grid using the trained RBMs."""
    # Start with random visible units
    visible = tf.random.uniform([1, 27])
    # Sample through the RBMs
    hidden1 = rbm1.sample_hidden(visible)
    hidden2 = rbm2.sample_hidden(hidden1)
    visible_recon = rbm1.sample_visible(rbm2.sample_visible(hidden2))
    # Reshape and decode the one-hot encoded grid
    grid = np.argmax(visible_recon.numpy().reshape(3, 3, 3), axis=2) + 1
    return grid

def complete_sudoku(rbm1, rbm2, partial_grid):
    """Complete a partially filled Sudoku grid."""
    # Flatten and one-hot encode the partial grid
    partial_flattened = (partial_grid.flatten() - 1).astype(int)
    partial_one_hot = np.eye(3)[partial_flattened].flatten()
    # Sample through the RBMs
    hidden1 = rbm1.sample_hidden(partial_one_hot[np.newaxis, :])
    hidden2 = rbm2.sample_hidden(hidden1)
    visible_recon = rbm1.sample_visible(rbm2.sample_visible(hidden2))
    # Reshape and decode the one-hot encoded grid
    completed_grid = np.argmax(visible_recon.numpy().reshape(3, 3, 3), axis=2) + 1
    return completed_grid

# Example usage
new_sudoku = generate_sudoku(rbm1, rbm2)
print("Generated Sudoku:\n", new_sudoku)

partial_grid = np.array([[1, 0, 0],
                         [0, 0, 0],
                         [0, 0, 0]])  # 0 represents empty cells
completed_sudoku = complete_sudoku(rbm1, rbm2, partial_grid)
print("Completed Sudoku:\n", completed_sudoku)